<a href="https://colab.research.google.com/github/shuntarotobari/qtest/blob/main/Growthtable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import pandas as pd

# Penn World Table 10.1のデータを読み込む
pwt = pd.read_stata('https://www.rug.nl/ggdc/docs/pwt100.dta')

# 必要な列と対象年の範囲を指定してデータをフィルタリング
countries = ['Australia', 'Austria', 'Belgium', 'Canada', 'Denmark', 'Finland', 'France', 'Germany', 'Greece', 'Iceland', 'Ireland', 'Italy', 'Japan', 'Netherlands', 'New Zealand', 'Norway', 'Portugal', 'Spain', 'Sweden', 'Switzerland', 'United Kingdom', 'United States']
data = pwt.loc[pwt['country'].isin(countries)][['year', 'countrycode', 'country', 'rgdpe', 'emp', 'cn', 'labsh']]
data = data.loc[(data['year'] >= 1990) & (data['year'] <= 2019)]

# 欠損値を0で埋める
data = data.fillna(0)

# 対象国のリストを作成
countries = data['countrycode'].unique()
country_names = list(data['country'].unique())

# 結果を格納するDataFrameを初期化
result = pd.DataFrame(index=country_names + ['Average'], columns=['Growth Rate', 'TFP Growth', 'Capital Deepening', 'TFP Share', 'Capital Share'])

for country, name in zip(countries, country_names):
    # 国ごとのデータを抽出
    country_data = data.loc[data['countrycode'] == country]

    # 必要な列を抽出
    y = country_data['rgdpe'] / country_data['emp']
    k = country_data['cn'] / country_data['emp']
    labor_share = country_data['labsh']

    # 計算
    alpha = 1 - labor_share
    growth_rate = (y / y.shift(1)).fillna(0)
    tfp_growth = ((y / k**alpha) / (y / k**alpha).shift(1)).fillna(0)
    capital_deepening = growth_rate - tfp_growth
    tfp_share = tfp_growth / growth_rate
    capital_share = 1 - tfp_share

    # 結果を代入
    result.loc[name, 'Growth Rate'] = growth_rate.mean()
    result.loc[name, 'TFP Growth'] = tfp_growth.mean()
    result.loc[name, 'Capital Deepening'] = capital_deepening.mean()
    result.loc[name, 'TFP Share'] = tfp_share.mean()
    result.loc[name, 'Capital Share'] = capital_share.mean()

# 平均値を計算して'Average'行に代入
result.loc['Average'] = result.mean()

print(result)

               Growth Rate TFP Growth Capital Deepening TFP Share  \
Australia          0.98243   0.957495          0.024934  0.975327   
Austria           0.987987   0.958348           0.02964  0.970513   
Belgium           0.984657   0.967506          0.017151  0.983642   
Canada            0.977876   0.963336          0.014539  0.988417   
Switzerland       0.984048   0.985625         -0.001577  1.003561   
Germany           0.985307   0.971762          0.013545  0.987756   
Denmark            0.99003   0.977625          0.012405  0.988931   
Spain             0.987321   0.947209          0.040111  0.959599   
Finland           0.986442   0.949172          0.037271  0.966588   
France            0.982256   0.976219          0.006036  0.994419   
United Kingdom    0.985805   0.983656          0.002149  0.997758   
Greece            0.982917   1.009018         -0.026101  1.027372   
Ireland            1.01489    0.90933           0.10556  0.905527   
Iceland            0.98504   1.046